# Detecting medical entities using the AWS SDK for Python (Boto)

## Configuration

In [ ]:
import boto3
import json
client = boto3.client(service_name='comprehendmedical', region_name='us-east-1')

## 1. Text analysis API operations

### 1.1 Detect entities (Version 2)
Examine unstructured clinical text to detect textual references to medical information such as medical condition, treatment, tests and results, and medications. This version uses a different model than the original Detect entities API, and there are a few changes in the output.

#### 1.1.1 Medication
Detects medication and dosage information on the patient.

In [ ]:
result = client.detect_entities(Text= 'aspirin is required 20 mg po daily for 2 times as tab')
entities = result['Entities'];
print(json.dumps(entities[0], indent=2))

#### 1.1.2 Multiple Entities
Detecting multiple entitie categories in the same text.

In [ ]:
result = client.detect_entities(Text= 'Pt is 87 yo woman, highschool teacher with past medical history that includes \
   - status post cardiac catheterization in April 2019. \
She presents today with palpitations and chest pressure. \
HPI : Sleeping trouble on present dosage of Clonidine. Severe Rash  on face and leg, slightly itchy. \
Meds : Vyvanse 50 mgs po at breakfast daily, \
            Clonidine 0.2 mgs -- 1 and 1 / 2 tabs po qhs \
HEENT : Boggy inferior turbinates, No oropharyngeal lesion. \
Lungs : clear. \
Heart : Regular rhythm. \
Skin :  Mild erythematous eruption to hairline. \
Follow-up as scheduled')
entities = result['Entities'];
print(json.dumps(entities, indent=2))

### 1.2 Detect PHI
Use the DetectPHI operation when you only want to detect Protected Health Information (PHI) data when scanning the clinical text. To detect all available entities in the clinical text use DetectEntitiesV2.

In [ ]:
result = client.detect_phi(Text= 'Patient is John Smith, a 48-year-old teacher and resident of Seattle, Washington.')
entities = result['Entities'];
print(json.dumps(entities, indent=2))

### 1.3 Text analysis batch APIs
Use Amazon Comprehend Medical to analyze medical text stored in an Amazon S3 bucket. Analyze up to 10 GB of documents in one batch. You use the console to create and manage batch analysis jobs, or use batch APIs to detect medical entities, including protected health information (PHI). The APIs start, stop, list, and describe ongoing batch analysis jobs.

For more information refer to https://docs.aws.amazon.com/comprehend-medical/latest/dev/textanalysis-batchapi.html

In [ ]:
# start a job
response = client.start_entities_detection_v2_job(
    InputDataConfig={
        'S3Bucket': '[BUCKET_NAME]',
        'S3Key': '[BUCKET_KEY]'
    },
    OutputDataConfig={
        'S3Bucket': '[BUCKET_NAME]',
        'S3Key': '[BUCKET_KEY'
    },
    DataAccessRoleArn='[IAM_ROLE]',
    JobName='CCDA-CM-Entity-Processing-Job',
    LanguageCode='en'
)
print(json.dumps(response, indent=2))

In [ ]:
# check job status
response = client.describe_entities_detection_v2_job(JobId='[JOB_ID]')
print(response)

## 2. Ontology linking APIs
Use Amazon Comprehend Medical to detect entities in clinical text and link those entities to concepts in standardized medical ontologies, including the RxNorm, ICD-10-CM, and SNOMED CT knowledge bases. You can perform analysis both on single files or as a batch analysis on large documents or multiple files stored in an Amazon Simple Storage Service (S3).

### 2.1 ICD-10-CM linking
When medical conditions are detected, InferICD10CM returns the matching ICD-10-CM codes and descriptions. The detected conditions are listed in descending order of confidence. The scores indicate the confidence in the accuracy of the entities matched to the concepts found in the text. Related information such as family history, signs, symptoms, and negation are recognized as traits. Additional information such as anatomical designations and acuity are listed as attributes.

In [ ]:
result = client.infer_icd10_cm(Text= 'The patient is a 71-year-old female patient of Dr. X. \
The patient presented to the emergency room last evening with approximately 7- to 8-day history of abdominal pain which has been persistent. \
She has had no nausea and vomiting, but has had persistent associated anorexia. \
She is passing flatus, but had some obstipation symptoms with the last bowel movement two days ago. \
She denies any bright red blood per rectum and no history of recent melena. \
Her last colonoscopy was approximately 5 years ago with Dr. Y. \
She has had no definite fevers or chills and no history of jaundice. The patient denies any significant recent weight loss.')
entities = result['Entities'];
print(json.dumps(entities, indent=2))

### 2.2 RxNorm linking
Amazon Comprehend Medical lists the top potentially matching RxCUIs for each medication that it detects in descending order by confidence score. Use the RxCUI codes for downstream analysis that is not possible with unstructured text. Related information such as strength, frequency, dose, dose form, and route of administration are listed as attributes in JSON format.

In [ ]:
result = client.infer_rx_norm(Text= 'fluoride topical ( fluoride 1.1 % topical gel ) 1 application Topically daily Brush onto teeth before bed time , spit , do not rinse, eat or drink for 20-30 minutes')
entities = result['Entities'];
print(json.dumps(entities, indent=2))